#**Extração de Características Visuais**

#Preparação do ambiente do Notebook





**Instalando Dependências**

In [ ]:
!pip install opencv-contrib-python==4.4.0.44

import cv2 as cv
import numpy as np
from moviepy.video.io.VideoFileClip import VideoFileClip
import pandas as pd
import pickle


**Obtendo Dados**

In [ ]:
! wget https://github.com/rudineigoularte/MBA_IABIGDATA/raw/main/videos2.tar.gz
! wget https://github.com/rudineigoularte/MBA_IABIGDATA/raw/main/videos3.tar.gz
! wget https://github.com/rudineigoularte/MBA_IABIGDATA/raw/main/videos4.tar.gz
! tar -xzvf videos2.tar.gz
! tar -xzvf videos3.tar.gz
! tar -xzvf videos4.tar.gz
! cp ./videos3/*.* ./videos2
! cp ./videos4/*.* ./videos2
! rm -r ./videos3
! rm -r ./videos4
! mkdir features

**Acesso ao Drive - para armazenamento de Arquivos**

O *pathname* precisa ser personalizado com o Drive do usuário deste Notebook. Atenção: alguns *pathnames* podem estar em Inglês enquanto outros em Português - verifique o seu caso. Crie um pasta no **seu** Drive onde os arquivos serão armazenados. Neste exemplo tal pasta é "MBA_Curso14_Dados".

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [7]:
cd '/content/drive/My Drive/MBA_Curso14_Dados/'

/content/drive/My Drive/MBA_Curso14_Dados


#<h1>Funções para Extração das Features Visuais</h1>

In [8]:
# ==========
# Extrator de Características: SIFT
# Recebe um frame como entrada e retorna uma lista com n vetores de características
# SIFT, cada um com dimensão 128.
## ==========
def extractSift(frame):

    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    sift = cv.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(gray_frame, None)
    return des


In [9]:
## ==========
# Pré-processamento dos dados: Redução de instâncias
# Para cada vídeo de entrada, presente na lista "videos", seleciona um conjunto
# de key-frames. Para cada key-frame são extraídas características SIFT.
#
# Assumindo videos com 24fps. A amostragem é de 1 frame a cada 3 seg, logo, com
# 24fps, implica 1 frame amostrado a cada 72 frames lidos. Adicionalmente pula-se
# os 10s iniciais e finais, aproximadamente, em cada vídeo -> aberturas/vinhetas/créditos.
#
# A variável de retorno "features" é um dicionário de tamanho n, com n sendo a 
# quantidade de vídeos. Cada posição do dicionário representa um vídeo x, e contém
# uma lista ("bag") que, por sua vez, contém os vetores de caracterísiticas SIFT
# dos key-frames selecionados do vídeo x.
## ==========
def getKeyFramesFeatures(videos):

    print("Extraindo caracterísitcas SIFT...")

    videoNumber = 0
    features = dict()
    for videoName in videos[0]:
      print(videoName)
      video = cv.VideoCapture("/content/videos2/"+videoName)

      fps = video.get(cv.CAP_PROP_FPS)      
      frame_count = int(video.get(cv.CAP_PROP_FRAME_COUNT))
      last = frame_count - 240 #para remover os últimos 10 segundos
      
      frameNumber = 1
      bag = []
      while video.isOpened():
          success, frame = video.read()
          if not success:
            break
          if success:
            if ((frameNumber >= 240) and (frameNumber < last)): #se o frame entre os 10s iniciais e os 10s finais de vídeo

              if (frameNumber % 72) == 0:  #amostragem de 1 frame a cada 3 seg
                bag.append(extractSift(frame))
          frameNumber += 1
      features[videoNumber] = bag
      videoNumber += 1

      video.release()

    return features

#Script para extrair características visuais

In [10]:
#Preparação dos dados
videos = pd.read_csv('/content/videos2/videos2.csv', sep = ',', header = None)

#Redução de instâncias/cracterísticas e extração de características
visualFeatures = getKeyFramesFeatures(videos)
#print("Tamanho visualFeatures: ", len(visualFeatures))

#Gravando as featrures em arquivo
#A pasta "features" deve existir dentro de "MBA_Curso14_Dados"
with open('./features2/visual_features.arq', 'wb') as arq_visual_features:
    pickle.dump(visualFeatures, arq_visual_features)


Extraindo caracterísitcas SIFT...
1801.mp4
-->FrameNumber:  4393
54176.mp4
-->FrameNumber:  4203
55274.mp4
-->FrameNumber:  4213
66915.mp4
-->FrameNumber:  3969
71264.mp4
-->FrameNumber:  3901
79333.mp4
-->FrameNumber:  10249
79925.mp4
-->FrameNumber:  2806
80285.mp4
-->FrameNumber:  5843
93128.mp4
-->FrameNumber:  2848
131239.mp4
-->FrameNumber:  3366
